# Organizando os Dados de Validação
#### Ref: https://www.kaggle.com/adviksharma/braille-images-for-english-characters
***

## Importação dos Pacotes
***

In [3]:
# pacote de gerenciamento de arquivos no sistema operacional
import os
# pacote de gerenciamento de arquivos
import shutil
# pacote de gerenciamento de conjunto de dados
import pandas as pd
# pacote de procedimentos numéricos utilizando matrizes 
import numpy as np
# pacote de visualização grafica
import matplotlib.pyplot as plt
# pacote de visão computacional
import cv2
# pacote de manipulação em imagens
from PIL import Image

## Organizando os Padrões dos Dados de Validação
***

In [1]:
# listando as imagens de validação
list_images = os.listdir('../files/images2/')

In [63]:
# carregando as imagens de teste em um array multidimensional
list_array = []
for i in range(0, len(list_images)):
    list_array.append(plt.imread('../files/images2/'+list_images[i]))

In [112]:
# padronizando a dimensão das imagens originais
list_array[12] = cv2.cvtColor(list_array[12], cv2.COLOR_GRAY2RGB)
list_array[14] = cv2.cvtColor(list_array[14], cv2.COLOR_GRAY2RGB)
list_array[20] = cv2.cvtColor(list_array[20], cv2.COLOR_GRAY2RGB)

In [116]:
# transformando a lista de imagens em um array multidimensional
list_array = np.array(list_array)

In [115]:
# modificando as dimensões das imagens
aux = []
for i in range(0, len(list_array)):
    aux.append(~cv2.cvtColor(cv2.resize(list_array[i], dsize = (24,33)), cv2.COLOR_BGR2GRAY))

aux = np.array(aux)
list_array = aux 

In [123]:
# salvando as imagens modificadas na pasta de validação
for i in range(0, len(list_array)):
    image = Image.fromarray(list_array[i].astype(np.uint8))
    image.save('../files/images2/'+list_images[i][0:-4]+'.tiff')

In [124]:
# removendo imagens no padrão antigo
for i in range(0, len(list_images)):
    os.remove('../files/images2/'+list_images[i])

In [125]:
# removendo arquivo compactado no padrão antigo
os.remove('../files/images2.zip')

## Aumento de Dados no Conjunto de Validação

In [126]:
def translate(image, Tx = 0, Ty = 0):
    '''
    Função que translada uma imagem na vertical e horizontal
    ''' 

    rows, cols = image.shape
    img = image.copy()
    matrix = np.array( [ [1, 0, Tx], [0, 1, Ty] ], dtype=np.float32)
    dst_img = cv2.warpAffine(img, matrix, (cols, rows))

    return dst_img

In [127]:
def rotation(image, theta=0):
    '''
    Função que rotaciona uma imagem para a esquerda e para direita
    '''
    
    rows, cols = image.shape
    img = image.copy()
    matrix = cv2.getRotationMatrix2D( (cols/2, rows/2),  theta , 1)
    dst_img = cv2.warpAffine(img, matrix, (cols, rows))

    return dst_img

In [128]:
# definindo intervalos úteis para variar as transformações geométricas
theta = np.arange(-10,10)
Tx = np.arange(-5, 5)
Ty = np.arange(-5, 5)
p_translate = np.concatenate((np.array([Tx, np.zeros(Ty.shape)]),np.array([np.zeros(Ty.shape), Ty])), axis = 1)
p_translate = np.concatenate((p_translate, np.array([Tx, Ty])), axis = 1)

In [133]:
# salvando imagens com transformações geométricas de rotação
for i in range(0, 26):
    for j in range(0, 20):
        img = Image.fromarray(rotation(image = list_array[i], theta = theta[j]))
        img.save('../files/images2/'+list_images[i][0:-4]+'-'+str(j+1)+'.tiff')

In [138]:
# salvando imagens com transformações geométricas de translação
for i in range(0, 26):
    for j in range(0, 30):
        img = Image.fromarray(translate(image = list_array[i], Tx = p_translate[0][j], Ty = p_translate[1][j]))
        img.save('../files/images2/'+list_images[i][0:-4]+'-'+str(j+21)+'.tiff')

## Salvando os Resultados Obtidos

In [139]:
# pegando as novas imagens de referencia
list_images = os.listdir('../files/images2/')

In [141]:
# carregando os rótulos das imagens em um array
alphabet = 'abcdefghijklmnopqrstuvwxyz'
labels = []
for i in range(0, len(list_images)):
    labels.append(alphabet.find(list_images[i][0:1])+1)

In [145]:
# organizando as dimensões dos dados
list_images = np.array(list_images)
labels = np.array(labels)
labels = np.reshape(labels, (len(labels), 1))
list_images = np.reshape(list_images, (len(list_images), 1))

In [148]:
# salvando as referências dos arquivos de imagens e seus rótulos em um dataframe
dataframe_array = np.concatenate((list_images, labels), axis=1)
dataframe = pd.DataFrame(dataframe_array, columns=['Path', 'Labels'])

In [150]:
# exportando o dataframe para um arquivos CSV
dataframe.to_csv('../csv_data/images2.csv')

In [4]:
# compactando imagens de teste no novo padrão desejado
shutil.make_archive('../files/images2', 'zip', '../files/images2')

'c:\\Users\\Alysson\\OneDrive\\Documents\\projetos-github\\braillingo\\files\\images2.zip'